# This notebook is created to test PyRTC with simulation paramaters of the 1.3m JCBT telescope.

In [1]:
#Import the basics
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib qt
import time
#Improt pyRTC Core classes
from pyRTC import *
from OOPAO.Telescope import Telescope
from OOPAO.Source import Source
from pyRTC.hardware.OOPAOInterface import OOPAOInterface

In [2]:
"""
Shared memory in python is a bit annoying, we are required to unlink it from the garbage collector
so that it will stick around in between runs, however sometime you can get into a situation where 
the SHM is not intialized properly. Usually you will see an error like: 
TypeError: buffer is too small for requested array

To reset a SHM you can run the following code. Note: it will throw some garbage collector errors.
"""
shm_names = ["wfs", "wfsRaw", "wfc", "wfc2D", "signal", "signal2D", "psfShort", "psfLong", "wfs.data","psf.data"] #list of SHMs to reset
clear_shms(shm_names)

Traceback (most recent call last):
  File "/usr/lib/python3.12/multiprocessing/resource_tracker.py", line 239, in main
    cache[rtype].remove(name)
KeyError: '/wfs'
Traceback (most recent call last):
  File "/usr/lib/python3.12/multiprocessing/resource_tracker.py", line 239, in main
    cache[rtype].remove(name)
KeyError: '/wfs_meta'
Traceback (most recent call last):
  File "/usr/lib/python3.12/multiprocessing/resource_tracker.py", line 239, in main
    cache[rtype].remove(name)
KeyError: '/wfs_gpu_handle'
Traceback (most recent call last):
  File "/usr/lib/python3.12/multiprocessing/resource_tracker.py", line 239, in main
    cache[rtype].remove(name)
KeyError: '/wfsRaw'
Traceback (most recent call last):
  File "/usr/lib/python3.12/multiprocessing/resource_tracker.py", line 239, in main
    cache[rtype].remove(name)
KeyError: '/wfsRaw_meta'
Traceback (most recent call last):
  File "/usr/lib/python3.12/multiprocessing/resource_tracker.py", line 239, in main
    cache[rtype].remove(

In [3]:
#Now we can read our YAML config file 
conf = utils.read_yaml_file("test_config_pyRTC_JCBT.yaml")

#And separate it into sections for each of our AO loop components
confLoop = conf["loop"]
confWFS = conf["wfs"]
confWFC = conf["wfc"]
confPSF = conf["psf"]
confSlopes = conf["slopes"]

print(confLoop)
print(confWFS)
print(confWFC)
print(confPSF)
print(confSlopes)

from test_parameter_file_JCBT import initializeParameterFile
param = initializeParameterFile()

"""
Create the OOPAO simulation interface object 
Running this cell will initialize the dm, wfs, psf, and slopes objects, 
but will not start their real time computations. This inialization includes
the creation of the Shared Memory Objects, and the simulation inialization.
"""
#from parameter_file.py import initializeParameterFile
#param = initializeParameterFile()

sim = OOPAOInterface(conf=conf, param=param)
#sim = OOPAOInterface(conf=conf, param=None)
wfs, dm, psf = sim.get_hardware()


test_M2C=np.load("test_M2C_JCBT_101_84.npy")
dm.setM2C(test_M2C)
dm.M2C

n_subaperture = param['nSubaperture']
#n_factor = param['nPixelPerSubap']
tel = Telescope(resolution           = param['resolution'],                          # resolution of the telescope in [pix]
                diameter             = param['diameter'],                                        # diameter in [m]        
                samplingTime         = param['samplingTime'],                                   # Sampling time in [s] of the AO loop
                centralObstruction   = param['centralObstruction'],                                      # Central obstruction in [%] of a diameter 
                display_optical_path = False,                                    # Flag to display optical path
                fov                  = 0 )
ngs=Source(optBand   = param['NGSband'],                         
           magnitude = param['NGSmagnitude'])

# combine the NGS to the telescope using '*' operator:
ngs*tel

tel.computePSF(zeroPaddingFactor = confPSF["zeropadding"])
crop_factor= confPSF["cropfactor"]
if crop_factor <= 0:
    raise ValueError("crop_factor must be > 0")

H, W = tel.PSF.shape

out_H = int(round(H / crop_factor))
out_W = int(round(W / crop_factor))

if out_H <= 0 or out_W <= 0:
    raise ValueError("crop_factor too large")

#True center (works for even & odd sizes)
cH = (H - 1) / 2
cW = (W - 1) / 2

# Start indices so that centers coincide
start_H = int(round(cH - (out_H - 1) / 2))
start_W = int(round(cW - (out_W - 1) / 2))

end_H = start_H + out_H
end_W = start_W + out_W

cropped_PSF=tel.PSF[start_H:end_H, start_W:end_W]

cropped_PSF

#tel_psf=np.load("PSF_JCBT_100_100.npy")
psf.setModelPSF(cropped_PSF)
#psf.psfShort

If jupyter pops up a warning:

```WARNING:root:Unable to adjust nice level.   Give your user sudo privledges without passowrd to use this feature.```

In case you don't have root access on the system you are using, consider the instructions given in automate_nice.txt

## Understanding some properties

### WFS

In [ ]:
wfs.height

In [ ]:
wfs.data

### DM

In [ ]:
dm.numActuators

In [ ]:
dm.numModes

In [ ]:
dm.affinity

In [ ]:
dm.correctionVector

In [ ]:
dm.correctionVector2D

In [ ]:
dm.flat

In [ ]:
dm.flatModal

In [ ]:
dm.actuatorStatus.shape

In [ ]:
dm.index_map

In [ ]:
dm.index_map.shape

In [ ]:
dm.floatMatrix

In [ ]:
dm.M2C

In [ ]:
dm.plot()

### PSF

In [ ]:
psf.imageShape

In [ ]:
psf.data

In [ ]:
psf.model

In [ ]:
plt.imshow(psf.data)

In [ ]:
psf.dark

In [ ]:
np.max(psf.data)

In [ ]:
psf.imageShape

# Back to RTC

In [4]:
slopes = SlopesProcess(conf=confSlopes)
loop = Loop(conf=confLoop)
dm.start()
wfs.start()
slopes.start()
#loop.loadIM(filename="saved_IM_test.npy")
psf.start()

### IMPORTANT: REAL-TIME VIEWING 

At this point, you can look at the shared memory objects to see what is happening in real time as we run the AO loop.

First, navigate on a terminal to the pyRTCView Folder: 

```cd INSTALLTION_LOCATION/pyRTC/pyRTCView```

Now run the viewer for whatever shared memory you would like to look at. For the default simulation run:

```
python pyRTCView.py wfs &
python pyRTCView.py wfc2D &
python pyRTCView.py signal2D &
python pyRTCView.py psfShort &
```

To set a static min and max, you can add the values in order (min, max) to the command line call like this:

```
python pyRTCView.py psfShort 0 65536 &
```
To see four important shared memory objects, run:
```
python pyRTCView_modified.py wfs wfc2D signal2D psfShort &
```

In [5]:
sim.removeAtmosphere()

In [6]:
dm.flatten()

In [7]:
psf.takeModelPSF() #Take a new model for the strehl calculation

loop.pokeAmp = 1e-8
loop.computeIM()

In [8]:
loop.start()

In [9]:
sim.addAtmosphere()